# Chatbot Documentation

This project was heavily inspired by the series of [YouTube videos](https://www.youtube.com/watch?v=dvOnYLDg8_Y&t=20s) by [sentdex](https://www.youtube.com/channel/UCfzlCWGWYyIQ0aLC5w48gBQ). In this notebook, I present comprehensive documentation of my experience with this series, which involved creating a chatbot with deep learning, Python and TensorFlow.

## Contents

1. Introduction & Collecting our Training Data
2. Data structure
3. Buffering dataset
4. Determining insert
5. Building database
6. Database to training data
7. Training a model
8. NMT Concepts and Parameters
9. Interacting with our Chatbot

# Introduction & Collecting our Training Data

The method used here in building this chatbot is designed to be generalizable for building chatbots that can be used for a diverse range of applications. A large differentiating factor between the different implementations of chatbots, will largely depend on the type of training data used to build the chatbot.  

As with all machine learning applications, one of the biggest obstacles is to collect the relevant data, and manipulate it to be useful for a given task. One of the most common data sets that people tend to use for building (relatively weak) chatbots is the open-source [Cornell movie database](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html), which contains 220,000 conversational exchanges between 10,300 pairs of movie characters. In addition to it being open-source, it contains conversational data from different movies, different characters, and different genders, which allows for a dataset that is fairly balanced to train on. However, a major draw-back is that there is a fairly limited amount of conversational exchanges.

In the era of deep learning, big data is really the fuel for our machine learning applications. As we are building a chatbot that utilizes deep learning, our model will necessarily be very data hungry. Therefore, the Cornell movie database discussed will be fairly limited in its size. In search of a larger corpus of conversational exchanges to train our chatbot, we thus turn to [Reddit](https://www.reddit.com/), a popular collection of online forums where people share news, stories, and other types of content, and users are encouraged to comment and discuss about everyones posts. The popularity of Reddit is immense. It was measured that almost 1.69 billion users had accessed the site in just the month of March 2018, alone ([source](https://www.statista.com/statistics/443332/reddit-monthly-visitors/)). 

To extract this conversational data from Reddit, we could have used the [Python Reddit API](https://praw.readthedocs.io/en/latest/), but it has some pretty strict limitations. This API unfortunately will not allow you to parse millions of rows without a longer period of time, or violating the Terms of Service. Instead we can use a Reddit post that provides a [data dump of 1.7 Billion Reddit Comments](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/?st=j9udbxta&sh=69e4fee7). This datafile is about 250 GB of data, when compressed. 

I will compare the performance of the chatbot on different sizes, and receny of reddit comment data which will be as follows:
1. A __small__ dataset of __1.4 GB__ collected from an __older__ time: __January 2012__
2. A __medium__ sized dataset of __9.1 GB__ collected from a more __recent__ time, __June 2018__
3. A __large__ dataset of ~ __250 GB__, from between __December 2005 to June 2018__

This data was collected from this [website](http://files.pushshift.io/reddit/comments/), where the datafiles are available in compressed form - allowing for relatively fast download speeds.

# Data Structure

If you observe the following [sample Reddit post](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/?sort=top) and scroll down to the comments, you can observe that the comments are arranged in a __tree-like structure__, where you have parent comments at the top, followed by child comments which are in response to the parent above it. What we will need to do is to pull these strings of comments apart, and then  pair them together in a parent-child (comment and reply) manner. This allows us to capture natural conversational exchanges between humans on the internet, which we can then provide as training data to our chatbot to mimic. 

In this section we begin by __building our database__ that will store our parent comments that are paired to their best child (reply) comments. The reason do this is because a lot of these files are way too big for us to read into RAM and create training files from. For now, to keep things relatively simple we will use __SQLite__ for our database. 

The datafiles we are using are stored in the __JSON__ format. Because of this, there is a lot of unnecessary data within each of the files. When building our database, we will extract the following data from our JSON files:
* Comment score (karma)
* The comment itself
* ...

Let's begin building that database using Python and SQLite!

In [3]:
import sqlite3  # For building our database
import json  # To parse our datafiles
from datetime import datetime

In [4]:
timeframe = '2012-01'  # Begin with our small dataset
sql_transaction = []  # Efficiently parse rows

In SQL, you ideally want to have a __big SQL transaction__ when possible. This is because you don't want to (for example) handle millions of rows by inserting them one by one if you don't have to, because that can be incredibly really inefficient. Instead you want to build up a big transaction and then perform it all at once - as this will be much faster to execute.

The code immediately below this writing will connect to, and create the database if it doesn't exist already.

In [5]:
connection = sqlite3.connect('{}.db'.format(timeframe))  # Connects to database
c = connection.cursor()  # Define cursor

Here, let's create a function that can help us store the parent_id, comment_id, the parent comment, the reply (comment), subreddit, the time, and then finally the score (votes) for the comments from the raw JSON files.

In [7]:
# Query function to extract data from raw JSON files
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT 
    PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit
    TEXT, unix INT, score INT)""")

In [8]:
# Creates the table, if it doesn't exist already:
if __name__ == '__main__':
    create_table()

# Buffering Data